## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

May. 4th May. 11th


In [3]:
filename = glob.glob('*redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,May-1-2025,Condo/Co-op,10804 SW 72 St #115,Miami,FL,33173.0,460000.0,4.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/10804-SW-72nd-...,MARMLS,A11749828,N,Y,25.699822,-80.368053


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [7]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [8]:
df2 = df[(df['SOLD DATE'] > one_week_ago) & (df['SOLD DATE'] < today)]

In [9]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-05-05    26
2025-05-06    21
2025-05-09    18
2025-05-07    15
2025-05-08    10
2025-05-10     1
Name: count, dtype: int64

In [10]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [11]:
df_top_ten = df2.head(10)

In [12]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
88,PAST SALE,2025-05-09,Condo/Co-op,16901 Collins Ave Ph 5603,Sunny Isles Beach,FL,33160.0,28641500.0,6.0,9.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11551985,N,Y,25.932764,-80.121036
76,PAST SALE,2025-05-05,Condo/Co-op,500 Alton #3003,Miami Beach,FL,33139.0,7000000.0,3.0,4.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/500-Alto...,MARMLS,A11633750,N,Y,25.775127,-80.141257
1,PAST SALE,2025-05-06,Condo/Co-op,5959 Collins Ave #1003,Miami Beach,FL,33140.0,6150000.0,3.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/5959-Col...,Beaches MLS,RX-11081791,N,Y,25.841880,-80.120969
31,PAST SALE,2025-05-09,Condo/Co-op,18975 Collins Ave #3901,Sunny Isles Beach,FL,33160.0,4375000.0,3.0,5.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11755214,N,Y,25.951740,-80.119755
78,PAST SALE,2025-05-08,Condo/Co-op,2201 Collins Ave Unit Bungalow 1,Miami Beach,FL,33139.0,3500000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/2201-Col...,MARMLS,A11664223,N,Y,25.797901,-80.127592
80,PAST SALE,2025-05-07,Condo/Co-op,900 Brickell Key Blvd #2603,Miami,FL,33131.0,3350000.0,4.0,5.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/900-Brickell-K...,MARMLS,A11655338,N,Y,25.769178,-80.185753
74,PAST SALE,2025-05-05,Condo/Co-op,3470 E Coast Ave Ph 101,Miami,FL,33137.0,2700000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/3470-E-Coast-A...,MARMLS,A11690007,N,Y,25.809553,-80.191593
23,PAST SALE,2025-05-09,Condo/Co-op,9703 Collins Ave #802,Bal Harbour,FL,33154.0,2550000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/9703-Col...,MARMLS,A11766679,N,Y,25.888840,-80.122478
91,PAST SALE,2025-05-06,Condo/Co-op,10201 Collins Ave #211,Bal Harbour,FL,33154.0,2300000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10201-Co...,MARMLS,A11476654,N,Y,25.895168,-80.123150
12,PAST SALE,2025-05-06,Condo/Co-op,161 Crandon Blvd #118,Key Biscayne,FL,33149.0,2250000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/161-Cra...,MARMLS,A11770512,N,Y,25.697741,-80.160753


In [12]:
# df_top_ten = df2.head(10)

## Collect Agent Information

In [13]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [14]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [15]:
response_df = pd.DataFrame(response_list)

In [16]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [17]:
df_top_ten = merged_df

In [18]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2
0,PAST SALE,2025-05-09,Condo/Co-op,16901 Collins Ave Ph 5603,Sunny Isles Beach,FL,33160.0,28641500.0,6.0,9.5,...,-80.121036,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,Chad Carroll,"Compass Florida, LLC",None,None,Chad Carroll,"Compass Florida, LLC",None,None
1,PAST SALE,2025-05-05,Condo/Co-op,500 Alton #3003,Miami Beach,FL,33139.0,7000000.0,3.0,4.0,...,-80.141257,https://www.redfin.com/FL/Miami-Beach/500-Alto...,Janet Hoyos,Douglas Elliman,None,None,Darin Tansey,Douglas Elliman,None,None
2,PAST SALE,2025-05-06,Condo/Co-op,5959 Collins Ave #1003,Miami Beach,FL,33140.0,6150000.0,3.0,4.5,...,-80.120969,https://www.redfin.com/FL/Miami-Beach/5959-Col...,Shane Aronson,Serhant,None,None,David Siddons,Douglas Elliman,Stefania Cambarau,Douglas Elliman
3,PAST SALE,2025-05-09,Condo/Co-op,18975 Collins Ave #3901,Sunny Isles Beach,FL,33160.0,4375000.0,3.0,5.5,...,-80.119755,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Daniel Tzinker,The Agency Florida LLC,None,None,Aleksandra Marzec,"The World Realty, Inc",None,None
4,PAST SALE,2025-05-08,Condo/Co-op,2201 Collins Ave Unit Bungalow 1,Miami Beach,FL,33139.0,3500000.0,2.0,2.5,...,-80.127592,https://www.redfin.com/FL/Miami-Beach/2201-Col...,Florian Jouin,One Sotheby's International Re,None,None,Florian Jouin,One Sotheby's International Re,None,None
5,PAST SALE,2025-05-07,Condo/Co-op,900 Brickell Key Blvd #2603,Miami,FL,33131.0,3350000.0,4.0,5.5,...,-80.185753,https://www.redfin.com/FL/Miami/900-Brickell-K...,Renata Galembeck Rohr,Douglas Elliman,None,None,Paulina Flores Mendez,DLS International Realty,None,None
6,PAST SALE,2025-05-05,Condo/Co-op,3470 E Coast Ave Ph 101,Miami,FL,33137.0,2700000.0,3.0,3.5,...,-80.191593,https://www.redfin.com/FL/Miami/3470-E-Coast-A...,Liz Hogan,"Compass Florida, LLC.",Kaitlyn Neumann,"Compass Florida, LLC",Gary Wong,Avanti Way Realty LLC,None,None
7,PAST SALE,2025-05-09,Condo/Co-op,9703 Collins Ave #802,Bal Harbour,FL,33154.0,2550000.0,2.0,2.0,...,-80.122478,https://www.redfin.com/FL/Bal-Harbour/9703-Col...,Dean Bloch,Douglas Elliman,None,None,Adva Golan,Jonathan Solomon Realty,None,None
8,PAST SALE,2025-05-06,Condo/Co-op,10201 Collins Ave #211,Bal Harbour,FL,33154.0,2300000.0,2.0,2.0,...,-80.123150,https://www.redfin.com/FL/Bal-Harbour/10201-Co...,Martina Colombres,Consultatio Realty LLC,None,None,Harel Bitton,Cromwell & Forbes Inc,None,None
9,PAST SALE,2025-05-06,Condo/Co-op,161 Crandon Blvd #118,Key Biscayne,FL,33149.0,2250000.0,3.0,3.0,...,-80.160753,https://www.redfin.com/FL/Key-Biscayne/161-Cra...,Dolores Urdapilleta,"Urdapilleta Real Estate, LLC",None,None,Todd Finlay PA,Douglas Elliman,None,None


## Current Week's Values

In [19]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

91


In [20]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$1,230,121


In [21]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$660


In [22]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$111,940,988


In [23]:
def load_last_week_data_df(filename='last_week_data.csv'):
    try:
        return pd.read_csv(filename)
    except FileNotFoundError:
        return None
    
last_week = load_last_week_data_df()

In [24]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
# previous_week_condo_sales = input()
previous_week_condo_sales = last_week['Condo Sales'].iloc[0]
print(previous_week_condo_sales)
print('-----------')

print('Input Previous Week Condo Average Sales Price:')
# previous_week_average_sales_price = input()
previous_week_average_sales_price = last_week['Average Sale Price'].iloc[0]
print(previous_week_average_sales_price)
print('-----------')

print('Input Previous Week Condo Average PSF:')
# previous_week_average_psf = input()
previous_week_average_psf = last_week['Average PSF'].iloc[0]
print(previous_week_average_psf)
print('-----------')

print('Input Previous Week Condo Sales total (ex: 198_000_000)')
# previous_week_condo_sales_total = input()
previous_week_condo_sales_total = last_week['Last Week Total Dollar Volume'].iloc[0]
print(previous_week_condo_sales_total)


## Clean Data

Input county name:
Miami-Dade
Input Previous Week Condo Sales Total (number units sold):
157
-----------
Input Previous Week Condo Average Sales Price:
871712
-----------
Input Previous Week Condo Average PSF:
568
-----------
Input Previous Week Condo Sales total (ex: 198_000_000)
136858723.0


In [25]:
last_week

,Condo Sales,Average Sale Price,Average PSF,Last Week Total Dollar Volume
0,157,871712,568,136858723.0


In [26]:
last_week['Last Week Total Dollar Volume'].iloc[0]

136858723.0

In [27]:
def save_current_week_data_df(df, filename='last_week_data.csv'):
    df.to_csv(filename, index=False)

# Create a dictionary of your current week data.
current_week_data = {
    "Condo Sales": current_week_condo_sales,
    "Average Sale Price": average_sale_price,
    "Average PSF": average_price_psf,
    "Last Week Total Dollar Volume": current_week_condo_sales_total
}

# Convert the dictionary into a DataFrame.
current_week_summary_df = pd.DataFrame([current_week_data])

# Save the DataFrame to a CSV file.
save_current_week_data_df(current_week_summary_df)

# Next week, load last week's data automatically.
last_week_df = load_last_week_data_df()
if last_week_df is not None:
    print("Last week's data:")
    print(last_week_df)
else:
    print("No previous data found.")

Last week's data:
   Condo Sales  Average Sale Price  Average PSF  Last Week Total Dollar Volume
0           91             1230121          660                    111940988.0


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [28]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [29]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [30]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [31]:
pd.set_option('display.max_columns',None)

In [32]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-05-09,Condo/Co-op,16901 Collins Ave Ph 5603,Sunny Isles Beach,FL,33160.0,28641500.0,6.0,9.5,JADE SIGNATURE CONDO,11661.0,NaN,2018.0,NaN,2456.0,18848.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11551985,N,Y,25.932764,-80.121036,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,Chad Carroll,"Compass Florida, LLC",None,None,Chad Carroll,"Compass Florida, LLC",None,None,1,orange


In [33]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [34]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-05-09,Condo/Co-op,16901 Collins Ave Ph 5603,Sunny Isles Beach,FL,33160.0,28641500.0,6.0,9.5,JADE SIGNATURE CONDO,11661.0,NaN,2018.0,NaN,2456.0,18848.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11551985,N,Y,25.932764,-80.121036,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,Chad Carroll,"Compass Florida, LLC",None,None,Chad Carroll,"Compass Florida, LLC",None,None,1,orange
1,PAST SALE,2025-05-05,Condo/Co-op,500 Alton #3003,Miami Beach,FL,33139.0,7000000.0,3.0,4.0,Five Park Miami Beach,2615.0,NaN,2024.0,NaN,2677.0,5230.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/500-Alto...,MARMLS,A11633750,N,Y,25.775127,-80.141257,https://www.redfin.com/FL/Miami-Beach/500-Alto...,Janet Hoyos,Douglas Elliman,None,None,Darin Tansey,Douglas Elliman,None,None,2,blue
2,PAST SALE,2025-05-06,Condo/Co-op,5959 Collins Ave #1003,Miami Beach,FL,33140.0,6150000.0,3.0,4.5,The Bath Club,3640.0,NaN,2005.0,NaN,1690.0,8312.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/5959-Col...,Beaches MLS,RX-11081791,N,Y,25.841880,-80.120969,https://www.redfin.com/FL/Miami-Beach/5959-Col...,Shane Aronson,Serhant,None,None,David Siddons,Douglas Elliman,Stefania Cambarau,Douglas Elliman,3,blue
3,PAST SALE,2025-05-09,Condo/Co-op,18975 Collins Ave #3901,Sunny Isles Beach,FL,33160.0,4375000.0,3.0,5.5,18975 COLLINS CONDO,2468.0,NaN,2019.0,NaN,1773.0,4838.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11755214,N,Y,25.951740,-80.119755,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Daniel Tzinker,The Agency Florida LLC,None,None,Aleksandra Marzec,"The World Realty, Inc",None,None,4,blue
4,PAST SALE,2025-05-08,Condo/Co-op,2201 Collins Ave Unit Bungalow 1,Miami Beach,FL,33139.0,3500000.0,2.0,2.5,2201 COLLINS AVE CONDO,1306.0,NaN,2009.0,NaN,2680.0,5010.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/2201-Col...,MARMLS,A11664223,N,Y,25.797901,-80.127592,https://www.redfin.com/FL/Miami-Beach/2201-Col...,Florian Jouin,One Sotheby's International Re,None,None,Florian Jouin,One Sotheby's International Re,None,None,5,blue
5,PAST SALE,2025-05-07,Condo/Co-op,900 Brickell Key Blvd #2603,Miami,FL,33131.0,3350000.0,4.0,5.5,ASIA CONDO,3398.0,NaN,2008.0,NaN,986.0,6318.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/900-Brickell-K...,MARMLS,A11655338,N,Y,25.769178,-80.185753,https://www.redfin.com/FL/Miami/900-Brickell-K...,Renata Galembeck Rohr,Douglas Elliman,None,None,Paulina Flores Mendez,DLS International Realty,None,None,6,blue
6,PAST SALE,2025-05-05,Condo/Co-op,3470 E Coast Ave Ph 101,Miami,FL,33137.0,2700000.0,3.0,3.5,TWO MIDTOWN MIAMI CONDO,3193.0,NaN,2007.0,NaN,846.0,3592.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/3470-E-Coast-A...,MARMLS,A11690007,N,Y,25.809553,-80.191593,https://www.redfin.com/FL/Miami/3470-E-Coast-A...,Liz Hogan,"Compass Florida, LLC.",Kaitlyn Neumann,"Compass Florida, LLC",Gary Wong,Avanti Way Realty LLC,None,None,7,blue
7,PAST SALE,2025-05-09,Condo/Co-op,9703 Collins Ave #802,Bal Harbour,FL,33154.0,2550000.0,2.0,2.0,BAL HARBOUR CENTER CONDO,1069.0,NaN,2011.0,NaN,2385.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/9703-Col...,MARMLS,A11766679,N,Y,25.888840,-80.122478,https://www.redfin.com/FL/Bal-Harbour/9703-Col...,Dean Bloch,Douglas Elliman,None,None,Adva Golan,Jonathan Solomon Realty,None,None,8,blue
8,PAST SALE,2025-05-06,Condo/Co-op,10201 Collins Ave #211,Bal Harbour,FL,33154.0,2300000.0,2.0,2.0,OCEANA BAL HARBOUR CONDO,1805.0,NaN,2016.0,NaN,1274.0,3863.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10

In [35]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent {county} County Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [36]:
m.save('index.html')

## Data snagger

In [37]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [38]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-05-09,Condo/Co-op,16901 Collins Ave Ph 5603,Sunny Isles Beach,FL,33160.0,28641500.0,6.0,9.5,JADE SIGNATURE CONDO,11661.0,NaN,2018.0,NaN,2456.0,18848.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11551985,N,Y,25.932764,-80.121036,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,Chad Carroll,"Compass Florida, LLC",None,None,Chad Carroll,"Compass Florida, LLC",None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [39]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MiamiDade_County_condo_sales_week_ending_05122025


# CREATE TEMPLATE 

In [40]:
muni_set = set(df_top_ten['CITY'])

In [41]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [42]:
df_top_ten.reset_index(inplace=True,drop=True)

In [43]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [44]:
top_sale

'JADE SIGNATURE CONDO at 16901 Collins Ave Ph 5603 in Sunny Isles Beach'

In [45]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [46]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [47]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-05-09,Condo/Co-op,16901 Collins Ave Ph 5603,Sunny Isles Beach,FL,33160.0,28641500.0,6.0,9.5,JADE SIGNATURE CONDO,11661.0,NaN,2018.0,NaN,2456.0,18848.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11551985,N,Y,25.932764,-80.121036,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,Chad Carroll,"Compass Florida, LLC",None,None,Chad Carroll,"Compass Florida, LLC",None,None,1,orange,JADE SIGNATURE CONDO at 16901 Collins Ave Ph 5...
1,PAST SALE,2025-05-05,Condo/Co-op,500 Alton #3003,Miami Beach,FL,33139.0,7000000.0,3.0,4.0,Five Park Miami Beach,2615.0,NaN,2024.0,NaN,2677.0,5230.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/500-Alto...,MARMLS,A11633750,N,Y,25.775127,-80.141257,https://www.redfin.com/FL/Miami-Beach/500-Alto...,Janet Hoyos,Douglas Elliman,None,None,Darin Tansey,Douglas Elliman,None,None,2,blue,Five Park Miami Beach at 500 Alton #3003 in Mi...
2,PAST SALE,2025-05-06,Condo/Co-op,5959 Collins Ave #1003,Miami Beach,FL,33140.0,6150000.0,3.0,4.5,The Bath Club,3640.0,NaN,2005.0,NaN,1690.0,8312.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/5959-Col...,Beaches MLS,RX-11081791,N,Y,25.841880,-80.120969,https://www.redfin.com/FL/Miami-Beach/5959-Col...,Shane Aronson,Serhant,None,None,David Siddons,Douglas Elliman,Stefania Cambarau,Douglas Elliman,3,blue,The Bath Club at 5959 Collins Ave #1003 in Mia...
3,PAST SALE,2025-05-09,Condo/Co-op,18975 Collins Ave #3901,Sunny Isles Beach,FL,33160.0,4375000.0,3.0,5.5,18975 COLLINS CONDO,2468.0,NaN,2019.0,NaN,1773.0,4838.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11755214,N,Y,25.951740,-80.119755,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Daniel Tzinker,The Agency Florida LLC,None,None,Aleksandra Marzec,"The World Realty, Inc",None,None,4,blue,18975 COLLINS CONDO at 18975 Collins Ave #3901...
4,PAST SALE,2025-05-08,Condo/Co-op,2201 Collins Ave Unit Bungalow 1,Miami Beach,FL,33139.0,3500000.0,2.0,2.5,2201 COLLINS AVE CONDO,1306.0,NaN,2009.0,NaN,2680.0,5010.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/2201-Col...,MARMLS,A11664223,N,Y,25.797901,-80.127592,https://www.redfin.com/FL/Miami-Beach/2201-Col...,Florian Jouin,One Sotheby's International Re,None,None,Florian Jouin,One Sotheby's International Re,None,None,5,blue,2201 COLLINS AVE CONDO at 2201 Collins Ave Uni...
5,PAST SALE,2025-05-07,Condo/Co-op,900 Brickell Key Blvd #2603,Miami,FL,33131.0,3350000.0,4.0,5.5,ASIA CONDO,3398.0,NaN,2008.0,NaN,986.0,6318.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/900-Brickell-K...,MARMLS,A11655338,N,Y,25.769178,-80.185753,https://www.redfin.com/FL/Miami/900-Brickell-K...,Renata Galembeck Rohr,Douglas Elliman,None,None,Paulina Flores Mendez,DLS International Realty,None,None,6,blue,ASIA CONDO at 900 Brickell Key Blvd #2603 in M...
6,PAST SALE,2025-05-05,Condo/Co-op,3470 E Coast Ave Ph 101,Miami,FL,33137.0,2700000.0,3.0,3.5,TWO MIDTOWN MIAMI CONDO,3193.0,NaN,2007.0,NaN,846.0,3592.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/3470-E-Coast-A...,MARMLS,A11690007,N,Y,25.809553,-80.191593,https://www.redfin.com/FL/Miami/3470-E-Coast-A...,Liz Hogan,"Compass Florida, LLC.",Kaitlyn Neumann,"Compass Florida, LLC",Gary Wong,Avanti Way Realty LLC,None,None,7,blue,TWO MIDTOWN MIAMI CONDO at 3470 E Coast Ave Ph...
7,PAST SALE,2025-05-09,Condo/Co-op,9703 Collins Ave #802,Bal Harbour,FL,33154.0,2550000.0,2.0,2.0,BAL HARBOUR CENTER CONDO,1069.0,NaN,2011.0,NaN,2385.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/9703-Col...,MARMLS,A11766679,N,Y,25.888840,-80

In [48]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}, condos, condo market

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: JADE SIGNATURE CONDO closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $2,250,000 to $28,641,500
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Bal Harbour, Key Biscayne, Miami, Miami Beach, Sunny Isles Beach, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 91 condo sales totaling $111,940,988 million from May. 4th to May. 11th. The previous week, brokers closed 157 condo sales totaling $136,858,723.

Last week’s units sold for an average of $1,230,121, higher than the $871,712 avera

In [49]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [50]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [51]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [52]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-05-09,Condo/Co-op,16901 Collins Ave Ph 5603,Sunny Isles Beach,FL,33160.0,28641500.0,6.0,9.5,JADE SIGNATURE CONDO,11661.0,NaN,2018.0,NaN,2456.0,18848.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11551985,N,Y,25.932764,-80.121036,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,Chad Carroll,"Compass Florida, LLC",None,None,Chad Carroll,"Compass Florida, LLC",None,None,1,orange,JADE SIGNATURE CONDO at 16901 Collins Ave Ph 5...
1,PAST SALE,2025-05-05,Condo/Co-op,500 Alton #3003,Miami Beach,FL,33139.0,7000000.0,3.0,4.0,Five Park Miami Beach,2615.0,NaN,2024.0,NaN,2677.0,5230.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/500-Alto...,MARMLS,A11633750,N,Y,25.775127,-80.141257,https://www.redfin.com/FL/Miami-Beach/500-Alto...,Janet Hoyos,Douglas Elliman,None,None,Darin Tansey,Douglas Elliman,None,None,2,blue,Five Park Miami Beach at 500 Alton #3003 in Mi...
2,PAST SALE,2025-05-06,Condo/Co-op,5959 Collins Ave #1003,Miami Beach,FL,33140.0,6150000.0,3.0,4.5,The Bath Club,3640.0,NaN,2005.0,NaN,1690.0,8312.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/5959-Col...,Beaches MLS,RX-11081791,N,Y,25.841880,-80.120969,https://www.redfin.com/FL/Miami-Beach/5959-Col...,Shane Aronson,Serhant,None,None,David Siddons,Douglas Elliman,Stefania Cambarau,Douglas Elliman,3,blue,The Bath Club at 5959 Collins Ave #1003 in Mia...
3,PAST SALE,2025-05-09,Condo/Co-op,18975 Collins Ave #3901,Sunny Isles Beach,FL,33160.0,4375000.0,3.0,5.5,18975 COLLINS CONDO,2468.0,NaN,2019.0,NaN,1773.0,4838.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11755214,N,Y,25.951740,-80.119755,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Daniel Tzinker,The Agency Florida LLC,None,None,Aleksandra Marzec,"The World Realty, Inc",None,None,4,blue,18975 COLLINS CONDO at 18975 Collins Ave #3901...
4,PAST SALE,2025-05-08,Condo/Co-op,2201 Collins Ave Unit Bungalow 1,Miami Beach,FL,33139.0,3500000.0,2.0,2.5,2201 COLLINS AVE CONDO,1306.0,NaN,2009.0,NaN,2680.0,5010.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/2201-Col...,MARMLS,A11664223,N,Y,25.797901,-80.127592,https://www.redfin.com/FL/Miami-Beach/2201-Col...,Florian Jouin,One Sotheby's International Re,None,None,Florian Jouin,One Sotheby's International Re,None,None,5,blue,2201 COLLINS AVE CONDO at 2201 Collins Ave Uni...
5,PAST SALE,2025-05-07,Condo/Co-op,900 Brickell Key Blvd #2603,Miami,FL,33131.0,3350000.0,4.0,5.5,ASIA CONDO,3398.0,NaN,2008.0,NaN,986.0,6318.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/900-Brickell-K...,MARMLS,A11655338,N,Y,25.769178,-80.185753,https://www.redfin.com/FL/Miami/900-Brickell-K...,Renata Galembeck Rohr,Douglas Elliman,None,None,Paulina Flores Mendez,DLS International Realty,None,None,6,blue,ASIA CONDO at 900 Brickell Key Blvd #2603 in M...
6,PAST SALE,2025-05-05,Condo/Co-op,3470 E Coast Ave Ph 101,Miami,FL,33137.0,2700000.0,3.0,3.5,TWO MIDTOWN MIAMI CONDO,3193.0,NaN,2007.0,NaN,846.0,3592.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/3470-E-Coast-A...,MARMLS,A11690007,N,Y,25.809553,-80.191593,https://www.redfin.com/FL/Miami/3470-E-Coast-A...,Liz Hogan,"Compass Florida, LLC.",Kaitlyn Neumann,"Compass Florida, LLC",Gary Wong,Avanti Way Realty LLC,None,None,7,blue,TWO MIDTOWN MIAMI CONDO at 3470 E Coast Ave Ph...
7,PAST SALE,2025-05-09,Condo/Co-op,9703 Collins Ave #802,Bal Harbour,FL,33154.0,2550000.0,2.0,2.0,BAL HARBOUR CENTER CONDO,1069.0,NaN,2011.0,NaN,2385.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/9703-Col...,MARMLS,A11766679,N,Y,25.888840,-80

In [53]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: JADE SIGNATURE CONDO closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $2,250,000 to $28,641,500
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Bal Harbour, Key Biscayne, Miami, Miami Beach, Sunny Isles Beach, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 91 condo sales totaling $111,940,988 million from May. 4th to May. 11th. The previous week, brokers closed 157 condo sales totaling $136,858,723.

Last week’s units sold for an average of $1,230,121, higher than the $871,712 avera

In [54]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Sunny-Isles-Beach/16901-Collins-Ave-33160/unit-5603/home/56743871


In [55]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Miami-Beach/500-Alton-Rd-33139/unit-3003/home/191911586


In [56]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Key-Biscayne/161-Crandon-Blvd-33149/unit-118/home/42922668


In [57]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Bal-Harbour/10201-Collins-Ave-33154/unit-211/home/56745839


In [58]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Miami-Beach/2201-Collins-Ave-33139/unit-Bungalow-1/home/187842283


In [59]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Miami/3470-E-Coast-Ave-33137/unit-101/home/43454205


In [60]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-05-09,Condo/Co-op,16901 Collins Ave Ph 5603,Sunny Isles Beach,FL,33160.0,28641500.0,6.0,9.5,JADE SIGNATURE CONDO,11661.0,NaN,2018.0,NaN,2456.0,18848.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11551985,N,Y,25.932764,-80.121036,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,Chad Carroll,"Compass Florida, LLC",None,None,Chad Carroll,"Compass Florida, LLC",None,None,1,orange,JADE SIGNATURE CONDO at 16901 Collins Ave Ph 5...
1,PAST SALE,2025-05-05,Condo/Co-op,500 Alton #3003,Miami Beach,FL,33139.0,7000000.0,3.0,4.0,Five Park Miami Beach,2615.0,NaN,2024.0,NaN,2677.0,5230.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/500-Alto...,MARMLS,A11633750,N,Y,25.775127,-80.141257,https://www.redfin.com/FL/Miami-Beach/500-Alto...,Janet Hoyos,Douglas Elliman,None,None,Darin Tansey,Douglas Elliman,None,None,2,blue,Five Park Miami Beach at 500 Alton #3003 in Mi...
2,PAST SALE,2025-05-06,Condo/Co-op,5959 Collins Ave #1003,Miami Beach,FL,33140.0,6150000.0,3.0,4.5,The Bath Club,3640.0,NaN,2005.0,NaN,1690.0,8312.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/5959-Col...,Beaches MLS,RX-11081791,N,Y,25.841880,-80.120969,https://www.redfin.com/FL/Miami-Beach/5959-Col...,Shane Aronson,Serhant,None,None,David Siddons,Douglas Elliman,Stefania Cambarau,Douglas Elliman,3,blue,The Bath Club at 5959 Collins Ave #1003 in Mia...
3,PAST SALE,2025-05-09,Condo/Co-op,18975 Collins Ave #3901,Sunny Isles Beach,FL,33160.0,4375000.0,3.0,5.5,18975 COLLINS CONDO,2468.0,NaN,2019.0,NaN,1773.0,4838.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11755214,N,Y,25.951740,-80.119755,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Daniel Tzinker,The Agency Florida LLC,None,None,Aleksandra Marzec,"The World Realty, Inc",None,None,4,blue,18975 COLLINS CONDO at 18975 Collins Ave #3901...
4,PAST SALE,2025-05-08,Condo/Co-op,2201 Collins Ave Unit Bungalow 1,Miami Beach,FL,33139.0,3500000.0,2.0,2.5,2201 COLLINS AVE CONDO,1306.0,NaN,2009.0,NaN,2680.0,5010.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/2201-Col...,MARMLS,A11664223,N,Y,25.797901,-80.127592,https://www.redfin.com/FL/Miami-Beach/2201-Col...,Florian Jouin,One Sotheby's International Re,None,None,Florian Jouin,One Sotheby's International Re,None,None,5,blue,2201 COLLINS AVE CONDO at 2201 Collins Ave Uni...
5,PAST SALE,2025-05-07,Condo/Co-op,900 Brickell Key Blvd #2603,Miami,FL,33131.0,3350000.0,4.0,5.5,ASIA CONDO,3398.0,NaN,2008.0,NaN,986.0,6318.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/900-Brickell-K...,MARMLS,A11655338,N,Y,25.769178,-80.185753,https://www.redfin.com/FL/Miami/900-Brickell-K...,Renata Galembeck Rohr,Douglas Elliman,None,None,Paulina Flores Mendez,DLS International Realty,None,None,6,blue,ASIA CONDO at 900 Brickell Key Blvd #2603 in M...
6,PAST SALE,2025-05-05,Condo/Co-op,3470 E Coast Ave Ph 101,Miami,FL,33137.0,2700000.0,3.0,3.5,TWO MIDTOWN MIAMI CONDO,3193.0,NaN,2007.0,NaN,846.0,3592.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/3470-E-Coast-A...,MARMLS,A11690007,N,Y,25.809553,-80.191593,https://www.redfin.com/FL/Miami/3470-E-Coast-A...,Liz Hogan,"Compass Florida, LLC.",Kaitlyn Neumann,"Compass Florida, LLC",Gary Wong,Avanti Way Realty LLC,None,None,7,blue,TWO MIDTOWN MIAMI CONDO at 3470 E Coast Ave Ph...
7,PAST SALE,2025-05-09,Condo/Co-op,9703 Collins Ave #802,Bal Harbour,FL,33154.0,2550000.0,2.0,2.0,BAL HARBOUR CENTER CONDO,1069.0,NaN,2011.0,NaN,2385.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/9703-Col...,MARMLS,A11766679,N,Y,25.888840,-80

## Time on Market Calculator

In [62]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 8, 1) ## List (Earlier) date
date2 = datetime(2025, 5, 5) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

277


## Clean CSV for Datawrapper Chart

In [63]:
chart_df = df_top_ten

In [64]:
chart_df = chart_df.fillna(" ")

In [65]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [66]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [67]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [68]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [69]:
chart_df['$/SQUARE FEET'].astype(float)

0    2456.0
1    2677.0
2    1690.0
3    1773.0
4    2680.0
5     986.0
6     846.0
7    2385.0
8    1274.0
9    1049.0
Name: $/SQUARE FEET, dtype: float64

In [70]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [71]:
csv_date_string = today.strftime("%m_%d_%Y")

In [72]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv",index=False)

In [73]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2025-05-09,16901 Collins Ave Ph 5603 in Sunny Isles Beach,"$28,641,500",6,9.5,"11,661",2018,"$2,456",MARMLS,A11551985,"Chad Carroll Compass Florida, LLC","Chad Carroll Compass Florida, LLC"
1,2025-05-05,500 Alton #3003 in Miami Beach,"$7,000,000",3,4.0,"2,615",2024,"$2,677",MARMLS,A11633750,Janet Hoyos Douglas Elliman,Darin Tansey Douglas Elliman
2,2025-05-06,5959 Collins Ave #1003 in Miami Beach,"$6,150,000",3,4.5,"3,640",2005,"$1,690",Beaches MLS,RX-11081791,Shane Aronson Serhant,David Siddons Douglas Elliman Stefania Cambara...
3,2025-05-09,18975 Collins Ave #3901 in Sunny Isles Beach,"$4,375,000",3,5.5,"2,468",2019,"$1,773",MARMLS,A11755214,Daniel Tzinker The Agency Florida LLC,"Aleksandra Marzec The World Realty, Inc"
4,2025-05-08,2201 Collins Ave Unit Bungalow 1 in Miami Beach,"$3,500,000",2,2.5,"1,306",2009,"$2,680",MARMLS,A11664223,Florian Jouin One Sotheby's International Re,Florian Jouin One Sotheby's International Re
5,2025-05-07,900 Brickell Key Blvd #2603 in Miami,"$3,350,000",4,5.5,"3,398",2008,$986,MARMLS,A11655338,Renata Galembeck Rohr Douglas Elliman,Paulina Flores Mendez DLS International Realty...
6,2025-05-05,3470 E Coast Ave Ph 101 in Miami,"$2,700,000",3,3.5,"3,193",2007,$846,MARMLS,A11690007,"Liz Hogan Compass Florida, LLC. Kaitlyn Neuman...",Gary Wong Avanti Way Realty LLC
7,2025-05-09,9703 Collins Ave #802 in Bal Harbour,"$2,550,000",2,2.0,"1,069",2011,"$2,385",MARMLS,A11766679,Dean Bloch Douglas Elliman,Adva Golan Jonathan Solomon Realty
8,2025-05-06,10201 Collins Ave #211 in Bal Harbour,"$2,300,000",2,2.0,"1,805",2016,"$1,274",MARMLS,A11476654,Martina Colombres Consultatio Realty LLC,Harel Bitton Cromwell & Forbes Inc
9,2025-05-06,161 Crandon Blvd #118 in Key Biscayne,"$2,250,000",3,3.0,"2,144",1981,"$1,049",MARMLS,A11770512,"Dolores Urdapilleta Urdapilleta Real Estate, L...",Todd Finlay PA Douglas Elliman
